In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
news_dataset = pd.read_csv('fake_news_detection.csv')


In [6]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [3]:
news_dataset.dropna(inplace=True)

In [4]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [5]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']


In [8]:
X = news_dataset.drop(columns='label', axis = 1)
Y = news_dataset['label']

In [14]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content =  stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [16]:
news_dataset['content'] = news_dataset['content'][1:1000].apply(stemming)

In [17]:
X= news_dataset['content'][1:1000].values
Y = news_dataset['label'][1:1000].values

In [18]:
X.shape

(999,)

In [19]:
Y.shape

(999,)

In [21]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [23]:
print(X)

  (0, 3813)	0.313187310136898
  (0, 1594)	0.2056024667997323
  (0, 1297)	0.6708491299813305
  (0, 858)	0.2851716822706189
  (0, 660)	0.2180551644431456
  (0, 510)	0.3956766581056075
  (0, 443)	0.16151014552832893
  (0, 350)	0.313187310136898
  (1, 3584)	0.4460229526885438
  (1, 2210)	0.46182513283259075
  (1, 1400)	0.3636530230043021
  (1, 1266)	0.3682233963412856
  (1, 738)	0.46182513283259075
  (1, 691)	0.32659857361910477
  (2, 3652)	0.25801380989332556
  (2, 3173)	0.37468502703730716
  (2, 2742)	0.39699645524645183
  (2, 1883)	0.25801380989332556
  (2, 1800)	0.39699645524645183
  (2, 1662)	0.37468502703730716
  (2, 642)	0.3365433885875969
  (2, 86)	0.39699645524645183
  (3, 3813)	0.48714719951921476
  (3, 3640)	0.3077276276408101
  (3, 3329)	0.2542930167746082
  :	:
  (996, 54)	0.23478247992460985
  (997, 3693)	0.22812757016218752
  (997, 3681)	0.3224923929456087
  (997, 2936)	0.3224923929456087
  (997, 2706)	0.30886728915729067
  (997, 2444)	0.30886728915729067
  (997, 1681)	0.341

In [24]:
X_train, X_test, Y_train, Y_test, = train_test_split(X, Y, test_size =0.3, stratify=Y, random_state=2)

In [25]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [26]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
print('Accuracy score of the training data : ', training_data_accuracy)
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9985693848354793
Accuracy score of the test data :  0.9166666666666666


In [29]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('News is Real')
else:
    print('News is Fake')

[0]
News is Real


User defined Values


In [46]:
test_data = pd.read_csv('fake_news_detection_test.csv')

In [47]:
test_data

,id,title,author,text,label
0,0,This is a fake news. No prove has been given f...,john harry,tech,0


In [48]:
test_data['content'] = test_data['author'] + ' '+test_data['title']


In [49]:
test_data

,id,title,author,text,label,content
0,0,This is a fake news. No prove has been given f...,john harry,tech,0,john harry This is a fake news. No prove has b...


In [50]:
X_test_data = test_data.drop(columns='label',axis= 1)
y_test_data = test_data['label']

In [51]:
print(X_test_data)
print(y_test_data)

   id                                              title      author  text  \
0   0  This is a fake news. No prove has been given f...  john harry  tech   

                                             content  
0  john harry This is a fake news. No prove has b...  
0    0
Name: label, dtype: int64


In [52]:
test_data['content'] = test_data['content'].apply(stemming)

In [53]:
test_data['content']

0    john harri fake news prove given prove
Name: content, dtype: object

In [54]:
X_test_data = test_data['content'].values
Y_test_data = test_data['label'].values

In [70]:
type(X_test_data)

numpy.ndarray

In [55]:
X_vect = vectorizer.transform(X_test_data)

In [56]:
X_vect

<1x3877 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [57]:
X_new = X_vect[0]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('News is Real')
else:
    print('News is Fake')

[1]
News is Fake


Direct user Entry

In [91]:
user_input = input("Enter News")
print(user_input)
news = []
news.append(user_input)
type(news)
news = stemming(news[0])
news = np.array([news])

news_vect = vectorizer.transform(news)
X_new = news_vect[0]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('News is Real')
else:
    print('News is Fake')

Israel is Becoming Pivotal to Chinaâ€™s Mid-Eastern Calculus http://journal-neo.org/2016/11/07/israel-is-becoming-pivotal-to-china-s-mid-eastern-calculus/
[1]
News is Fake
